In [2]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
import collections
from functools import partial
from tqdm import tqdm

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use('deepretina')

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#### Load problem specific starter code

In [3]:
cd 'cs228/pa3'

/Users/lmcintosh/code/ipython-notebooks/classes/cs228/pa3


In [4]:
ls

cluster_graph.py   cluster_graph.pyc  do_pa3.py          do_pa3.pyc         factors.py         factors.pyc        images.mat         ldpc36-128.mat     ldpc36-1600.mat


In [5]:
import do_pa3

Doing part (a): Should see 0.0, 0.0, >0.0
(0.0, 0.0, 0.77378093749999988)
Doing part (c)


In [6]:
def test_channel_noise():
    p_test = [0.01, 0.05, 0.1, 0.5, 1.0]
    y = np.round(np.random.rand(100,20)).astype('int')
    fraction_flipped = []
    for p in p_test:
        yhat = do_pa3.applyChannelNoise(y, p)
        fraction_flipped.append(np.sum(abs(yhat - y))/(1.0*np.prod(y.shape)))
    for idp, p in enumerate(p_test):
        print('Actual p = %0.2f, empirical p = %0.2f' %(p, fraction_flipped[idp]))
#     print(fraction_flipped)
#     print(p_test)
#     return np.mean(np.array(p_test) - np.array(fraction_flipped))

In [7]:
test_channel_noise()

Actual p = 0.01, empirical p = 0.01
Actual p = 0.05, empirical p = 0.06
Actual p = 0.10, empirical p = 0.11
Actual p = 0.50, empirical p = 0.48
Actual p = 1.00, empirical p = 1.00


# Part (a)

In [8]:
do_pa3.do_part_a()

(0.0, 0.0, 0.77378093749999988)


# Part (b)

Implemented loopy BP

# Part (c)

In [9]:
from do_pa3 import *
from factors import *
from cluster_graph import *

In [ ]:
G, H = loadLDPC('ldpc36-128.mat')
p = 0.2
N = G.shape[1]
x = np.zeros((N, 1), dtype='int32')
y = encodeMessage(x, G)
##############################################################
# To do: your code starts here

# apply noise on y
yhat = applyChannelNoise(y, p)

# initialize graph
Graph = constructClusterGraph(yhat, H, p)

# initialize neighbors and messages
for i,f in enumerate(Graph.factor):
    Graph.nbr[i] = f.scope

    #for v in Graph.nbr[i]:
    for v in Graph.nbr[i]:
        to_var = Factor(scope=[v], card=[2], val=np.array([0.5, 0.5]))
        to_fac = Factor(scope=[v], card=[2], val=np.array([0.5, 0.5]))
        Graph.messages['var %d, fac %d' %(v,i)] = to_fac
        Graph.messages['fac %d, var %d' %(i,v)] = to_var

# initialize varToCliques
for var_i in range(len(Graph.varToCliques)):
    for fac_j,neighbors in enumerate(Graph.nbr):
        if var_i in neighbors:
            Graph.varToCliques[var_i].append(fac_j)


iterations = 50
Graph.runParallelLoopyBP(iterations)

# collect probabilities that bits = 1
marginals = [Graph.estimateMarginalProbability(i)[1] for i in range(len(yhat))]

#     import pdb
#     pdb.set_trace()

# plot it
plt.scatter(range(len(yhat)), marginals, s=10, color='k')

  0%|          | 0/50 [00:00<?, ?it/s]

In [1]:
len(Graph.messages)

NameError: name 'Graph' is not defined

In [79]:
Graph.messages

{'fac 326, var 198': Factor [unnamed]:
 Factor object with scope [198] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'var 136, fac 55': Factor [unnamed]:
 Factor object with scope [136] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'fac 114, var 72': Factor [unnamed]:
 Factor object with scope [72] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'fac 41, var 4': Factor [unnamed]:
 Factor object with scope [4] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'var 249, fac 95': Factor [unnamed]:
 Factor object with scope [249] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'fac 172, var 44': Factor [unnamed]:
 Factor object with scope [44] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'fac 158, var 30': Factor [unnamed]:
 Factor object with scope [30] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'fac 84, var 9': Factor [unnamed]:
 Factor object with scope [9] and corresponding cardinalities [2]
 CPD:
 [ 0.5  0.5], 'var 99, 

In [73]:
Graph.varToCliques

[[76, 92, 115, 128],
 [38, 43, 107, 129],
 [9, 63, 115, 130],
 [39, 53, 112, 131],
 [41, 51, 60, 132],
 [42, 56, 89, 133],
 [28, 60, 119, 134],
 [17, 51, 66, 135],
 [67, 116, 119, 136],
 [84, 88, 91, 137],
 [27, 76, 97, 138],
 [32, 39, 82, 139],
 [104, 105, 110, 140],
 [2, 61, 72, 141],
 [23, 58, 76, 142],
 [47, 99, 104, 143],
 [1, 58, 86, 144],
 [27, 48, 93, 145],
 [25, 90, 98, 146],
 [75, 103, 115, 147],
 [4, 19, 127, 148],
 [22, 90, 113, 149],
 [5, 55, 57, 150],
 [15, 29, 113, 151],
 [111, 117, 127, 152],
 [40, 76, 111, 153],
 [5, 72, 96, 154],
 [13, 27, 65, 155],
 [1, 94, 102, 156],
 [36, 68, 74, 157],
 [56, 85, 112, 158],
 [43, 89, 94, 159],
 [3, 29, 58, 160],
 [26, 42, 60, 161],
 [42, 47, 96, 162],
 [0, 17, 70, 163],
 [66, 71, 96, 164],
 [4, 24, 50, 165],
 [24, 29, 101, 166],
 [45, 73, 112, 167],
 [3, 83, 106, 168],
 [4, 56, 64, 169],
 [12, 71, 125, 170],
 [97, 106, 124, 171],
 [80, 98, 108, 172],
 [5, 34, 87, 173],
 [18, 30, 87, 174],
 [5, 7, 68, 175],
 [63, 69, 112, 176],
 [21,

In [75]:
Graph.runParallelLoopyBP(2)

> /Users/lmcintosh/code/ipython-notebooks/classes/cs228/pa3/cluster_graph.py(99)runParallelLoopyBP()
-> self.messages[key] = self.messages[key].marginalize_all_but([i])
(Pdb) [s]
[0]
(Pdb) self.messages[key]
Factor ((((((((([unnamed] * [unnamed]) * [unnamed]) * [unnamed]) * [unnamed]) * ((([unnamed] * [unnamed]) * [unnamed]) * [unnamed])) * ((([unnamed] * [unnamed]) * [unnamed]) * [unnamed])) * ((([unnamed] * [unnamed]) * [unnamed]) * [unnamed])) * ((([unnamed] * [unnamed]) * [unnamed]) * [unnamed])) * ((([unnamed] * [unnamed]) * [unnamed]) * [unnamed])):
Factor object with scope [128, 98, 35, 214, 86, 207] and corresponding cardinalities [2 2 2 2 2 2]
CPD:
[[[[[[  2.98023224e-08   2.98023224e-08]
     [  2.98023224e-08   2.98023224e-08]]

    [[  2.98023224e-08   2.98023224e-08]
     [  2.98023224e-08   2.98023224e-08]]]


   [[[  2.98023224e-08   2.98023224e-08]
     [  2.98023224e-08   2.98023224e-08]]

    [[  2.98023224e-08   2.98023224e-08]
     [  2.98023224e-08   2.98023224e-08

BdbQuit: 

In [80]:
Graph.estimateMarginalProbability(0)

> /Users/lmcintosh/code/ipython-notebooks/classes/cs228/pa3/cluster_graph.py(151)estimateMarginalProbability()
-> for i,f in enumerate(self.varToCliques[var]):
(Pdb) self.varToCliques[0]
[76, 92, 115, 128]
(Pdb) self.factor[76]
Factor parity:
Factor object with scope [0, 10, 14, 25, 116, 204] and corresponding cardinalities [2, 2, 2, 2, 2, 2]
CPD:
[ 1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.
  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  1.  0.
  1.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.
  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.]

(Pdb) self.factor[92]
Factor parity:
Factor object with scope [0, 62, 68, 87, 101, 220] and corresponding cardinalities [2, 2, 2, 2, 2, 2]
CPD:
[ 1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.
  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  1.  0.
  1.  0.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.
  1.  0.  0.  1.  1.  0. 

BdbQuit: 